In [1]:
import pandas as pd

In [2]:

df = pd.read_csv('../input/pima-indians-diabetes/pima-indians-diabetes.csv',header=None)
#df['target']=digits.target
df

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
X = df.drop(df.columns[8],axis='columns')
y = df[8]
X

,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


**OUTLIER REMOVAL**

In [4]:
for i in range(8):
    df=df[abs(((df.iloc[:,i]-df.iloc[:,i].mean())/df.iloc[:,i].std()))<3]
    i+=1
df

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
5,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
knn=KNeighborsClassifier(n_neighbors=3)
model = make_pipeline(StandardScaler(), knn)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7359307359307359

import numpy as np
from sklearn.decomposition import PCA
pca = PCA(0.95)
X_pca=pca.fit_transform(X)

 print(pca.explained_variance_ratio_)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=1)
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=10)
model = make_pipeline(StandardScaler(), knn)

model.fit(X_train,y_train)

model.score(X_test,y_test)

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.7922077922077922

In [8]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7835497835497836

In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=15, random_state=0)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7748917748917749

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores.mean()

0.7162719633307869

In [11]:
from sklearn.ensemble import BaggingClassifier
bag=BaggingClassifier( base_estimator=DecisionTreeClassifier(),
                       n_estimators=100,
                       max_samples=0.8,
                       oob_score=True,
                       random_state=0
                     )
bag.fit(X_train,y_train)
bag.oob_score_

0.7337057728119181

In [12]:
bag.score(X_test,y_test)

0.7835497835497836

In [13]:
def bag(model):
    bag_model = BaggingClassifier(
    base_estimator=model, 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
    )
    scores = cross_val_score(bag_model, X, y, cv=5)
    return(scores.mean())

In [14]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.7604617604617605

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
bag(clf)

0.7734997029114675

In [16]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000)
bag(model)

0.7721840251252015

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3)
model = make_pipeline(StandardScaler(), knn)
bag(model)

0.7434767846532553